In [1]:
!pip install --upgrade kaggle-environments
!pip install --upgrade pygame

  Using cached pygame-2.6.1-cp312-cp312-win_amd64.whl.metadata (13 kB)
   ---------------------------------------- 0.0/10.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.6 MB ? eta -:--:--
    --------------------------------------- 0.3/10.6 MB ? eta -:--:--
    --------------------------------------- 0.3/10.6 MB ? eta -:--:--
   - -------------------------------------- 0.5/10.6 MB 670.4 kB/s eta 0:00:16
   - -------------------------------------- 0.5/10.6 MB 670.4 kB/s eta 0:00:16
   -- ------------------------------------- 0.8/10.6 MB 657.8 kB/s eta 0:00:15
   -- ------------------------------------- 0.8/10.6 MB 657.8 kB/s eta 0:00:15
   --- ------------------------------------ 1.0/10.6 MB 671.0 kB/s eta 0:00:15
   --- ------------------------------------ 1.0/10.6 MB 671.0 kB/s eta 0:00:15
   ---- ----------------------------------- 1.3/10.6 MB 677.8 kB/s eta 0:00:14
   ---- ------------------

In [2]:
%%writefile Chess_bot__Step_0002__invisible.py

import time,random
from Chessnut import Game


piece_sym = ['K', 'k', 'Q', 'q', 'R', 'r', 'B', 'b', 'N', 'n', 'P', 'p']
piece_val = [0, 0, 100, 100, 50, 50, 30, 30, 30, 30, 10, 10]
piece_val_dict = dict(zip(piece_sym, piece_val))


def evaluate_position(board):
    total_score = 0
    for square in range(64):
        piece = board.get_piece(square)
        if piece != ' ':
            piece_value = piece_val_dict.get(piece.upper(), 0)
            if piece.isupper(): 
                total_score += piece_value # White piece
            else: 
                total_score -= piece_value # Black piece
    return total_score
    

def make_move(move, time_to_start_thinking, is_execute_print=False):
    duration = round(time.time() - time_to_start_thinking,3)
    if is_execute_print == True:
        print(f'[{move}, {duration}]')
    return move


def chess_bot__Step_0002__invisible(obs):
    time_start = time.time()
    game = Game(obs.board)
    moves = list(game.get_moves())

    evas, moves_eva, eva_init = [],[], evaluate_position(game.board)

    # Remove 'random' queen
    
    for move in moves:
        g = Game(obs.board)
        g.apply_move(move)  
        eva_after = evaluate_position(g.board)
        moves_eva.append({
            'move': move,
            'max' : eva_after - eva_init}
        )
        evas.append(eva_after)

    if len(moves_eva) >= 1:
        moves_eva = sorted(moves_eva, key=lambda x: x['max'], reverse=True)
        max_move = moves_eva[0] if moves_eva != None and len(moves_eva) >= 1 else None
        if max_move != None and type(max_move['move'])=='string':
            if max_move['max'] == 100 : return make_move(time_start, max_move['move'])
            if max_move['max'] == 50  : return make_move(time_start, max_move['move'])

    # some step Italian game, debut

    if 'e1g1' in moves: return make_move('e1g1',time_start)
    if 'e8g8' in moves: return make_move('e8g8',time_start)
    if 'e1c1' in moves: return make_move('e1c1',time_start)
    if 'e8c8' in moves: return make_move('e8c8',time_start)
    
    if obs.mark == 'white':
        if obs.step == 0 and 'e2e4' in moves: return make_move('e2e4',time_start)
        if obs.step == 2 and 'f1c4' in moves: return make_move('f1c4',time_start)
        if obs.step == 4 and 'g1f3' in moves: return make_move('g1f3',time_start)

    # exact copy of Bovard Doerschuk-Tiberi
    # 1. Check a subset of moves for checkmate
    for move in moves:
        g = Game(obs.board)
        g.apply_move(move)
        if g.status == Game.CHECKMATE:
            return make_move(move,time_start)

    # 2. Check for captures
    for move in moves:
        if game.board.get_piece(Game.xy2i(move[2:4])) != ' ':
            #print(f'\n{move}\n')
            return make_move(move,time_start)

    # 3. Check for queen promotions
    for move in moves:
        if "q" in move.lower():
            #print(f'\n{move}\n')
            return make_move(move,time_start)

    # 4. Random move if no checkmates or captures

    move = random.choice(moves)
    
    return make_move(move,time_start)

Writing Chess_bot__Step_0002__invisible.py


In [3]:
%%writefile Chess_bot__Step_0003__invisible.py

import time,random
from Chessnut import Game


piece_sym = ['K', 'k', 'Q', 'q', 'R', 'r', 'B', 'b', 'N', 'n', 'P', 'p']
piece_val = [0, 0, 100, 100, 50, 50, 30, 30, 30, 30, 10, 10]
piece_val_dict = dict(zip(piece_sym, piece_val))


def evaluate_position(board):
    total_score = 0
    for square in range(64):
        piece = board.get_piece(square)
        if piece != ' ':
            piece_value = piece_val_dict.get(piece.upper(), 0)
            if piece.isupper(): 
                total_score += piece_value # White piece
            else: 
                total_score -= piece_value # Black piece
    return total_score
    

def make_move(move, time_to_start_thinking, is_execute_print=False):
    duration = round(time.time() - time_to_start_thinking,3)
    if is_execute_print == True:
        print(f'[{move}, {duration}]')
    return move


def panic_mode(game, fen):
    # Aaaahhhh! Oh no, I'm running out of time!
    moves = game.get_moves()
    for move in moves:
        game.apply_move(move)
        if game.status == Game.CHECKMATE:
            return move
        game.set_fen(fen)
    return random.choice(moves)


def Chess_bot__Step_0003__invisible(obs):
    time_start = time.time()
    game = Game(obs.board)
    moves = list(game.get_moves())

    
    game = Game(fen=obs.board)  # Intialize the game state with the current FEN
    if obs.remainingOverageTime < 2:
        return panic_mode(game, obs.board)

    # Remove 'random' queen

    evas, moves_eva, eva_init = [],[], evaluate_position(game.board)
    
    for move in moves:
        g = Game(obs.board)
        g.apply_move(move)  
        eva_after = evaluate_position(g.board)
        moves_eva.append({
            'move': move,
            'max' : eva_after - eva_init}
        )
        evas.append(eva_after)

    if len(moves_eva) >= 1:
        moves_eva = sorted(moves_eva, key=lambda x: x['max'], reverse=True)
        max_move = moves_eva[0] if moves_eva != None and len(moves_eva) >= 1 else None
        if max_move != None:
            if max_move['max'] == 100 : 
                print(max_move)
                return make_move(max_move['move'],time_start)
            if max_move['max'] == 50  : 
                print(max_move)
                return make_move(max_move['move'],time_start)

    
    # don't give up the queen

    
    # evas, moves_eva, bad_moves = [],[],[]

    # for move in moves:
    #     g = Game(obs.board)
    #     g.apply_move(move)  
    #     eva_after = evaluate_position(g.board)
    #     moves_eva.append({
    #         'fen'            : g.get_fen(),
    #         'opp_moves'      : list(g.get_moves()),
    #         'move'           : move,
    #         'current_eva'    : eva_after - eva_init,
    #         'answer_opp_min' : 0}
    #     )
    #     evas.append(eva_after)

    #     print(len(oppo_moves))
    #     # print( 'qnt moves oppo = ',len(moves_eva['opp_moves']), oppo_moves)

    # opp_mins = []
        
    # for e in moves_eva:
    #     g = Game(e['fen'])
    #     opp_moves = list(g.get_moves())
    #     # afters = []
    #     # for move in opp_moves:
    #     #     g = Game(e['fen'])
    #     #     g.apply_move(move)
    #     #     eva_after = evaluate_position(g.board)
    #     #     afters.append(eva_after)
    #     # e['min'] = sorted(afters)[0]
    #     opp_mins.append(e['answer_opp_min'])
    
    # exact copy of Bovard Doerschuk-Tiberi
    # 1. Check a subset of moves for checkmate
    for move in moves:
        g = Game(obs.board)
        g.apply_move(move)
        if g.status == Game.CHECKMATE:
            return make_move(move,time_start)

    # 2. Check for captures
    for move in moves:
        if game.board.get_piece(Game.xy2i(move[2:4])) != ' ':
            #print(f'\n{move}\n')
            return make_move(move,time_start)

    # 3. Check for queen promotions
    for move in moves:
        if "q" in move.lower():
            #print(f'\n{move}\n')
            return make_move(move,time_start)

    # 4. Random move if no checkmates or captures

    move = random.choice(moves)
    
    return make_move(move,time_start)

Writing Chess_bot__Step_0003__invisible.py


In [4]:
%%writefile Chess_bot__MiniMax_ab_Pruning__Thomas_in_KL.py

import time
import random
import Chessnut
from Chessnut import Game


piece_sym = ['K', 'k', 'Q', 'q', 'R', 'r', 'B', 'b', 'N', 'n', 'P', 'p']
piece_val = [0, 0, 100, 100, 50, 50, 30, 30, 30, 30, 10, 10]
piece_val_dict = dict(zip(piece_sym, piece_val))


def evaluate_position(board):
    total_score = 0
    for square in range(64):
        piece = board.get_piece(square)
        if piece != ' ':
            piece_value = piece_val_dict.get(piece.upper(), 0)
            if piece.isupper(): 
                total_score += piece_value # White piece
            else: 
                total_score -= piece_value # Black piece
    return total_score


# Minimax algorithm with alpha-beta pruning to evaluate and choose the best move for the bot.
def minimax(game, depth, maximizing_player, alpha=float('-inf'), beta=float('inf')):
    if depth == 0 or game.status in [Game.CHECKMATE, Game.STALEMATE]:
        return evaluate_position(game.board), None
    moves = list(game.get_moves())
    best_move = None

    if maximizing_player:
        max_eval = float('-inf')
        for move in moves:
            g = Game(game.get_fen())
            g.apply_move(move)
            eval_score, _ = minimax(g, depth - 1, False, alpha, beta)
            if eval_score > max_eval:
                max_eval = eval_score
                best_move = move
            alpha = max(alpha, eval_score)
            if beta <= alpha:
                break  # Beta cut-off
        return max_eval, best_move
    else:
        min_eval = float('inf')
        for move in moves:
            g = Game(game.get_fen())
            g.apply_move(move)
            eval_score, _ = minimax(g, depth - 1, True, alpha, beta)
            if eval_score < min_eval:
                min_eval = eval_score
                best_move = move
            beta = min(beta, eval_score)
            if beta <= alpha:
                break  # Alpha cut-off
        return min_eval, best_move

default_fen = 'rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1'
    
def Chess_bot__MiniMax_ab_Pruning__Thomas_in_KL(obs):
    # Get FEN string from the Board object
    game = Game(obs.board)
    moves = list(game.get_moves())
    
    # 1. Try to detect checkmate
    for move in moves:
        g = Game(obs.board)
        g.apply_move(move)
        if g.status == Game.CHECKMATE:
            return move

    # 2. Check for queen promotions
    for move in moves:
        if "q" in move.lower():
            return move

    # 3. Use minimax algorithm with a depth of 3 to find the best move
    _, best_move = minimax(game, 1, True)
    return best_move if best_move else random.choice(moves)

Writing Chess_bot__MiniMax_ab_Pruning__Thomas_in_KL.py


In [5]:
%%writefile Chess_bot__starter_submission__Muhammad_Qasim_Shabbir.py

import Chessnut
from Chessnut import Game

def Chess_bot__starter_submission__Muhammad_Qasim_Shabbir(obs):
    
    # 0. Parse the current board state and generate legal moves using Chessnut library
    game = Game(obs.board)
    moves = list(game.get_moves())
    
    # Piece-square tables for positional evaluation
    piece_square_table = {
        'P': [  # Pawn
            0, 5, 5, 0, 5, 10, 50, 0,
            0, 5, 5, 0, 5, 10, 50, 0,
            0, 5, 10, 10, 10, 20, 50, 0,
            0, 10, 10, 20, 20, 30, 50, 0,
            0, 10, 20, 30, 30, 40, 50, 0,
            0, 10, 30, 40, 40, 50, 50, 0,
            0, 5, 5, 0, 5, 10, 50, 0,
            0, 0, 0, 0, 0, 0, 0, 0
        ],
        'N': [  # Knight
            -50, -40, -30, -30, -30, -30, -40, -50,
            -40, -20, 0, 0, 0, 0, -20, -40,
            -30, 0, 10, 15, 15, 10, 0, -30,
            -30, 5, 15, 20, 20, 15, 5, -30,
            -30, 0, 15, 20, 20, 15, 0, -30,
            -30, 5, 10, 15, 15, 10, 5, -30,
            -40, -20, 0, 5, 5, 0, -20, -40,
            -50, -40, -30, -30, -30, -30, -40, -50
        ],
        'B': [  # Bishop
            -20, -10, -10, -10, -10, -10, -10, -20,
            -10, 5, 0, 0, 0, 0, 5, -10,
            -10, 10, 10, 10, 10, 10, 10, -10,
            -10, 0, 10, 10, 10, 10, 0, -10,
            -10, 5, 5, 10, 10, 5, 5, -10,
            -10, 0, 5, 10, 10, 5, 0, -10,
            -10, 0, 0, 0, 0, 0, 0, -10,
            -20, -10, -10, -10, -10, -10, -10, -20
        ],
        'R': [  # Rook
            0, 0, 0, 5, 5, 0, 0, 0,
            -5, 0, 0, 0, 0, 0, 0, -5,
            -5, 0, 0, 0, 0, 0, 0, -5,
            -5, 0, 0, 0, 0, 0, 0, -5,
            -5, 0, 0, 0, 0, 0, 0, -5,
            -5, 0, 0, 0, 0, 0, 0, -5,
            5, 10, 10, 10, 10, 10, 10, 5,
            0, 0, 0, 5, 5, 0, 0, 0
        ],
        'Q': [  # Queen
            -20, -10, -10, -5, -5, -10, -10, -20,
            -10, 0, 0, 0, 0, 0, 0, -10,
            -10, 0, 5, 5, 5, 5, 0, -10,
            -5, 0, 5, 5, 5, 5, 0, -5,
            0, 0, 5, 5, 5, 5, 0, -5,
            -10, 5, 5, 5, 5, 5, 0, -10,
            -10, 0, 5, 0, 0, 0, 0, -10,
            -20, -10, -10, -5, -5, -10, -10, -20
        ],
        'K': [  # King (early game)
            -30, -40, -40, -50, -50, -40, -40, -30,
            -30, -40, -40, -50, -50, -40, -40, -30,
            -30, -40, -40, -50, -50, -40, -40, -30,
            -30, -40, -40, -50, -50, -40, -40, -30,
            -20, -30, -30, -40, -40, -30, -30, -20,
            -10, -20, -20, -20, -20, -20, -20, -10,
            20, 20, 0, 0, 0, 0, 20, 20,
            20, 30, 10, 0, 0, 10, 30, 20
        ]
    }

    # Function to evaluate material value
    def material_value(piece):
        values = {'P': 1, 'N': 3, 'B': 3, 'R': 5, 'Q': 9, 'K': 0}
        return values.get(piece.upper(), 0)

    # Function to evaluate a move
    def evaluate_move(move):
        g = Game(obs.board)
        g.apply_move(move)

        # Material gain (captures)
        captured_piece = game.board.get_piece(Game.xy2i(move[2:4]))
        material_gain = material_value(captured_piece) if captured_piece != ' ' else 0
        
        # Positional value (basic example with piece-square tables)
        start_sq = Game.xy2i(move[:2])
        end_sq = Game.xy2i(move[2:4])
        piece = game.board.get_piece(start_sq).upper()
        positional_value = piece_square_table.get(piece, [0]*64)[end_sq] - \
                           piece_square_table.get(piece, [0]*64)[start_sq]
        
        # Combining material gain and positional value
        return material_gain + positional_value

    # 1. Check for checkmate moves
    for move in moves:
        g = Game(obs.board)
        g.apply_move(move)
        if g.status == Game.CHECKMATE:
            return move

    # 2. Evaluate and rank moves based on strategies
    scored_moves = [(move, evaluate_move(move)) for move in moves]
    scored_moves.sort(key=lambda x: x[1], reverse=True)

    # 3. Additional Strategies

    # 3.1. Center control strategy
    def center_control(board):
        center_squares = ['d4', 'd5', 'e4', 'e5']
        score = 0
        for square in center_squares:
            piece = board.get_piece(Game.xy2i(square))
            if piece != ' ':
                score += material_value(piece)
        return score
    
    # 3.2. King safety: prioritizing moves that protect the king
    def king_safety(board):
        king_square = board.find_king(1 if board.turn == 'w' else 0)
        safe_squares = [king_square]
        score = 0
        for sq in safe_squares:
            piece = board.get_piece(sq)
            if piece != ' ':
                score += material_value(piece)
        return score
    
    # 3.3. Threat detection: prioritize moves that threaten the opponent's high-value pieces
    def threat_detection(board):
        score = 0
        for move in moves:
            end_sq = Game.xy2i(move[2:4])
            opponent_piece = board.get_piece(end_sq)
            if opponent_piece != ' ' and material_value(opponent_piece) >= 5:
                score += material_value(opponent_piece)
        return score

    # Ranking moves based on the new strategies
    best_move = max(scored_moves, key=lambda x: x[1])[0]
    return best_move

Writing Chess_bot__starter_submission__Muhammad_Qasim_Shabbir.py


In [6]:
%%writefile Chess_bot__initial_agent__William_Guesdon.py

import time
from Chessnut import Game
import random

def chess_bot__initial_agent__William_Guesdon(obs):
    """
    A heuristic-based chess bot that prioritizes:
    - Checkmates
    - Captures
    - Promotions
    - Random moves
    Args:
        obs: Object with 'board' representing board state as FEN string
    Returns:
        Move in UCI notation
    """
    game = Game(obs.board)
    moves = list(game.get_moves())
    random.shuffle(moves)  # Randomize moves to add variation

    # Prioritize checkmates
    for move in moves[:10]:
        g = Game(obs.board)
        g.apply_move(move)
        if g.status == Game.CHECKMATE:
            return move

    # Check for captures
    for move in moves:
        if game.board.get_piece(Game.xy2i(move[2:4])) != ' ':
            return move

    # Check for promotions
    for move in moves:
        if "q" in move.lower():  # Queen promotion
            return move

    # Default to random move
    return random.choice(moves)

Writing Chess_bot__initial_agent__William_Guesdon.py


In [7]:
%%writefile Chess_bot__LittleDeepBlue_0_5_5__William_Guesdon.py

from Chessnut import Game
import time
import random

PIECE_VALUES = {'p': 100, 'n': 320, 'b': 330, 'r': 500, 'q': 900, 'k': 20000}

def evaluate_board(game):
    """
    Evaluate the board position from the perspective of the current player.
    Positive scores are good for the current player, negative for the opponent.
    """
    board_str = str(game.board)
    score = 0
    for char in board_str:
        if char.isalpha():
            if char.isupper():
                score += PIECE_VALUES[char.lower()]
            else:
                score -= PIECE_VALUES[char]
    return score

def minimax(game, depth, alpha, beta, maximizing_player, start_time, time_limit):
    if time.time() - start_time > time_limit:
        return None, None
    if depth == 0 or game.status >= 2:
        return evaluate_board(game), None
    moves = list(game.get_moves())
    if not moves:
        return evaluate_board(game), None
    best_move = None
    if maximizing_player:
        max_eval = float('-inf')
        for move in moves:
            # Time check
            if time.time() - start_time > time_limit:
                break
            new_game = Game(str(game))
            new_game.apply_move(move)
            eval_score, _ = minimax(new_game, depth - 1, alpha, beta, False, start_time, time_limit)
            if eval_score is None:
                continue
            if eval_score > max_eval:
                max_eval = eval_score
                best_move = move
            alpha = max(alpha, eval_score)
            if beta <= alpha:
                break
        return max_eval, best_move
    else:
        min_eval = float('inf')
        for move in moves:
            # Time check
            if time.time() - start_time > time_limit:
                break
            new_game = Game(str(game))
            new_game.apply_move(move)
            eval_score, _ = minimax(new_game, depth - 1, alpha, beta, True, start_time, time_limit)
            if eval_score is None:
                continue
            if eval_score < min_eval:
                min_eval = eval_score
                best_move = move
            beta = min(beta, eval_score)
            if beta <= alpha:
                break
        return min_eval, best_move

def heuristic_chess_bot(obs):
    game = Game(obs.board)
    moves = list(game.get_moves())
    best_score = float('-inf')
    best_move = None
    for move in moves:
        move_score = 0
        from_square = move[:2]
        to_square = move[2:4]
        piece_moved = game.board.get_piece(Game.xy2i(from_square))
        target_piece = game.board.get_piece(Game.xy2i(to_square))
        # Add score for capturing a piece
        if target_piece != ' ':
            move_score += PIECE_VALUES[target_piece.lower()] - PIECE_VALUES[piece_moved.lower()]
        # Add a small bonus for developing pieces (moving from the back rank)
        if piece_moved.lower() in ['n', 'b', 'q']:
            if (piece_moved.isupper() and from_square[1] == '1') or (piece_moved.islower() and from_square[1] == '8'):
                move_score += 10
        # Check for promotions
        if "q" in move.lower():
            move_score += PIECE_VALUES['q']
        # Prefer castling
        if move in ['e1g1', 'e1c1', 'e8g8', 'e8c8']:
            move_score += 50
        if move_score > best_score:
            best_score = move_score
            best_move = move
    if best_move:
        return best_move
    else:
        return random.choice(moves)

def hybrid_chess_bot(obs):
    game = Game(obs.board)
    start_time = time.time()
    time_limit = 0.09  # Set per-move time limit to 0.09 seconds
    best_move = None
    moves = list(game.get_moves())
    if not moves:
        return "0000"  # No legal moves
    depth = 1
    try:
        # Limit depth to ensure completion within time limit
        while depth <= 2:
            elapsed_time = time.time() - start_time
            if elapsed_time >= time_limit:
                break
            current_time_limit = time_limit - elapsed_time
            eval_score, move = minimax(game, depth, float('-inf'), float('inf'), True, start_time, time_limit)
            if move:
                best_move = move
            depth += 1
    except Exception:
        pass
    if best_move:
        return best_move
    else:
        # Time is running out or no move found, use heuristic or random move
        elapsed_time = time.time() - start_time
        if elapsed_time < time_limit:
            return heuristic_chess_bot(obs)
        else:
            return random.choice(moves)


def chess_bot__LittleDeepBlue_0_5_5__William_Guesdon(obs, config):
    try:
        return random.choice([hybrid_chess_bot(obs),heuristic_chess_bot(obs)])
    except Exception:
        return random.choice(list(Game(obs.board).get_moves()))

Writing Chess_bot__LittleDeepBlue_0_5_5__William_Guesdon.py


In [8]:
%%writefile Chess_bot__LittleDeepBlue_0_10_4_debug__William_Guesdon.py

from Chessnut import Game
import time
import random

DEBUG = True

# Piece values for evaluation
PIECE_VALUES = {'p': 1, 'n': 3, 'b': 3, 'r': 5, 'q': 9, 'k': 0}

def log_message(message):
    """Helper function to print messages if DEBUG is True."""
    if DEBUG:
        current_time = time.time()
        print(f"[{current_time:.2f}] {message}")

def enhanced_heuristic_with_time_limit(obs, per_move_time, remaining_time):
    """
    Enhanced heuristic evaluation with adjusted scoring to prioritize captures and protect endangered pieces.
    Includes detailed logging via log_message.
    """
    game = Game(obs.board)
    moves = list(game.get_moves())
    random.shuffle(moves)  # Randomize moves to add variation
    log_message(f"Available moves: {moves}")

    best_score = float('-inf')
    best_move = None
    start_time = time.time()

    for move in moves:
        move_start = time.time()
        elapsed = move_start - start_time
        remaining = remaining_time - elapsed
        if remaining <= 0:
            log_message("Time limit for heuristic evaluation reached. Stopping.")
            break  # Stop evaluating if total time runs out

        log_message(f"Evaluating move: {move}. Elapsed: {elapsed:.2f}s, Remaining: {remaining:.2f}s")

        if elapsed >= per_move_time:
            log_message(f"Skipping move {move} due to per-move time limit.")
            continue  # Skip move if per-move time limit is exceeded

        move_score = 0
        from_square = move[:2]
        to_square = move[2:4]

        # Simulate the move
        new_game = Game(str(game))
        new_game.apply_move(move)

        # Immediate checkmate detection
        if new_game.status == Game.CHECKMATE:
            log_message(f"Checkmate detected with move: {move}. Returning immediately.")
            return move

        # Piece and target evaluation
        piece_moved = game.board.get_piece(Game.xy2i(from_square))
        target_piece = game.board.get_piece(Game.xy2i(to_square))

        # Add score for captures (material gain)
        if target_piece != ' ':
            # Increase capture bonus based on the value of the captured piece
            capture_value = PIECE_VALUES[target_piece.lower()]
            move_score += capture_value * 100  # Amplify the importance of captures
            log_message(f"Move {move} captures {target_piece}. Capture value: {capture_value}. Updated score: {move_score}")

            # Encourage favorable exchanges
            piece_value = PIECE_VALUES.get(piece_moved.lower(), 0)
            net_gain = capture_value - piece_value
            move_score += net_gain * 50  # Encourage exchanges where net gain is positive
            log_message(f"Net gain from exchange: {net_gain}. Updated score: {move_score}")

        # Prioritize pawn promotion
        if 'q' in move.lower():
            move_score += PIECE_VALUES['q'] * 100  # Increase bonus for promotion
            log_message(f"Move {move} promotes to queen. Updated score: {move_score}")

        # Center control bonus (e4, d4, e5, d5)
        if to_square in ['d4', 'e4', 'd5', 'e5']:
            move_score += 20
            log_message(f"Move {move} controls center. Updated score: {move_score}")

        # Check detection
        if new_game.status == Game.CHECK:
            move_score += 30
            log_message(f"Move {move} gives check. Updated score: {move_score}")

        # Evaluate if any own pieces are under attack after the move
        opponent_moves = list(new_game.get_moves())
        for opp_move in opponent_moves:
            opp_target_square = opp_move[2:4]
            own_piece = new_game.board.get_piece(Game.xy2i(opp_target_square))
            if own_piece != ' ' and own_piece.isupper():  # Assuming bot plays as White
                # Penalize based on the value of the endangered piece
                endangered_piece_value = PIECE_VALUES.get(own_piece.lower(), 0)
                move_score -= endangered_piece_value * 100  # Adjust penalty multiplier as needed
                log_message(f"Own piece {own_piece} at {opp_target_square} is endangered. Penalizing move. Updated score: {move_score}")

        # Update the best move if the score is better
        if move_score > best_score:
            best_score = move_score
            best_move = move
            log_message(f"New best move found: {best_move}. Best score updated to: {best_score}")

        move_end = time.time()
        log_message(f"Move {move} evaluation completed in {move_end - move_start:.2f}s.")

    log_message(f"Best move chosen: {best_move} with final score: {best_score}")
    return best_move if best_score > float('-inf') else None


def Chess_bot__LittleDeepBlue_0_10_4_debug__William_Guesdon(obs, config):
    """
    Combined chess agent with total and per-move time limits.
    """
    try:
        start_time = time.time()
        total_time_limit = 8.0
        per_move_time_limit = 0.1

        log_message("Starting agent evaluation")

        while True:
            elapsed_time = time.time() - start_time
            remaining_time = total_time_limit - elapsed_time

            log_message(f"Total elapsed: {elapsed_time:.2f}s, Remaining time: {remaining_time:.2f}s")

            if remaining_time <= 0.1:  # Grace period to avoid timeout
                log_message("Time is almost up. Defaulting to a random move.")
                break

            # Step 1: Use enhanced heuristic with per-move and remaining time limits
            move = enhanced_heuristic_with_time_limit(obs, per_move_time=per_move_time_limit, remaining_time=remaining_time)
            if move:
                log_message(f"Heuristic selected move: {move}")
                return move

        # Step 2: Default to a random move if time is exhausted
        game = Game(obs.board)
        random_move = random.choice(list(game.get_moves()))
        log_message(f"Random move chosen: {random_move}")
        return random_move

    except Exception as e:
        # Emergency fallback to a random move
        log_message(f"Error in agent: {e}")
        game = Game(obs.board)
        fallback_move = random.choice(list(game.get_moves()))
        log_message(f"Fallback random move chosen: {fallback_move}")
        return fallback_move

Writing Chess_bot__LittleDeepBlue_0_10_4_debug__William_Guesdon.py


In [9]:
%%writefile Chess_bot__with_scoring__0I1.py

from Chessnut import Game

def Chess_bot__with_scoring__0I1(obs):
    """
    Compact chess bot that prioritizes checkmates, captures, and key moves.
    """
    game = Game(obs.board)
    moves = list(game.get_moves())
    if not moves:
        return None
        
    # Piece values for capture evaluation. Same as in the game
    values = {'P':1, 'N':3, 'B':3, 'R':5, 'Q':9, 'K':0} # King does not matter here
    
    best_score = -9999
    best_moves = []
    
    for move in moves:
        score = 0
        # Try move and check for checkmate. This one is first priority.
        test_game = Game(obs.board)
        test_game.apply_move(move)
        if test_game.status == Game.CHECKMATE:
            return move
            
        # If caoture exists, evaluate captures
        captured = game.board.get_piece(Game.xy2i(move[2:4]))
        if captured != ' ':
            score = values.get(captured.upper(), 0)
            
        # Bonus for queen promotions
        if 'q' in move.lower():
            score += 8 # P = 1, Q = 9
            
        # Track best moves
        if score > best_score:
            best_score = score
            best_moves = [move]
        elif score == best_score:
            best_moves.append(move)
            
    # Return a random best move
    return best_moves[0] if best_moves else random.choice(moves)

Writing Chess_bot__with_scoring__0I1.py


In [10]:
%%writefile Chess_bot__Small_Snail__LennartHaupts.py

import random
from Chessnut import Game

# Value map for chess pieces
value_map = {
    " ": 0,
    "p": 10,   "P": 10,  # Pawns
    "n": 30,   "N": 30,  # Knights
    "b": 35,   "B": 35,  # Bishops
    "r": 50,   "R": 50,  # Rooks
    "q": 90,   "Q": 90,  # Queens
    "k": 2000, "K": 2000
}

standard_board_map = {
    "a8": -1, "b8": 2, "c8": 3, "d8": 3, "e8": 3, "f8": 3, "g8": 2, "h8": -1,
    "a7": 0, "b7": 1, "c7": 1, "d7": 1, "e7": 1, "f7": 1, "g7": 1, "h7": 0,
    "a6": 0, "b6": 3, "c6": 4, "d6": 4, "e6": 4, "f6": 4, "g6": 3, "h6": 0,
    "a5": 0, "b5": 3, "c5": 5, "d5": 8, "e5": 8, "f5": 5, "g5": 3, "h5": 0,
    "a4": 0, "b4": 3, "c4": 5, "d4": 8, "e4": 8, "f4": 5, "g4": 3, "h4": 0,
    "a3": 0, "b3": 3, "c3": 4, "d3": 4, "e3": 4, "f3": 4, "g3": 3, "h3": 0,
    "a2": 0, "b2": 1, "c2": 1, "d2": 1, "e2": 1, "f2": 1, "g2": 1, "h2": 0,
    "a1": -1, "b1": 2, "c1": 3, "d1": 3, "e1": 3, "f1": 3, "g1": 2, "h1": -1
    }

pawn_board_map = {
    "a8": 8, "b8": 8, "c8": 8, "d8": 8, "e8": 8, "f8": 8, "g8": 8, "h8": 8,
    "a7": 8, "b7": 8, "c7": 8, "d7": 8, "e7": 8, "f7": 8, "g7": 8, "h7": 8,
    "a6": 0, "b6": 0, "c6": 4, "d6": 4, "e6": 4, "f6": 4, "g6": 0, "h6": 0,
    "a5": 0, "b5": 2, "c5": 5, "d5": 8, "e5": 8, "f5": 5, "g5": 2, "h5": 0,
    "a4": 0, "b4": 2, "c4": 5, "d4": 8, "e4": 8, "f4": 5, "g4": 2, "h4": 0,
    "a3": 0, "b3": 0, "c3": 4, "d3": 4, "e3": 4, "f3": 4, "g3": 0, "h3": 0,
    "a2": 8, "b2": 8, "c2": 8, "d2": 8, "e2": 8, "f2": 8, "g2": 8, "h2": 8,
    "a1": 8, "b1": 8, "c1": 8, "d1": 8, "e1": 8, "f1": 8, "g1": 8, "h1": 8
}

king_board_map = {
    "a8": -2, "b8": -2, "c8": -3, "d8": -4, "e8": -4, "f8": -3, "g8": -2, "h8": -2,
    "a7": -2, "b7": -3, "c7": -4, "d7": -5, "e7": -5, "f7": -4, "g7": -3, "h7": -2,
    "a6": -3, "b6": -4, "c6": -5, "d6": -6, "e6": -6, "f6": -5, "g6": -4, "h6": -3,
    "a5": -4, "b5": -5, "c5": -6, "d5": -7, "e5": -7, "f5": -6, "g5": -5, "h5": -4,
    "a4": -4, "b4": -5, "c4": -6, "d4": -7, "e4": -7, "f4": -6, "g4": -5, "h4": -4,
    "a3": -3, "b3": -4, "c3": -5, "d3": -6, "e3": -6, "f3": -5, "g3": -4, "h3": -3,
    "a2": -2, "b2": -3, "c2": -4, "d2": -5, "e2": -5, "f2": -4, "g2": -3, "h2": -2,
    "a1": -2, "b1": -2, "c1": -3, "d1": -4, "e1": -4, "f1": -3, "g1": -2, "h1": -2
}

rook_board_map = {
    "a8": 5, "b8": 6, "c8": 7, "d8": 8, "e8": 8, "f8": 7, "g8": 6, "h8": 5,
    "a7": 4, "b7": 5, "c7": 6, "d7": 7, "e7": 7, "f7": 6, "g7": 5, "h7": 4,
    "a6": 3, "b6": 4, "c6": 5, "d6": 6, "e6": 6, "f6": 5, "g6": 4, "h6": 3,
    "a5": 3, "b5": 4, "c5": 5, "d5": 6, "e5": 6, "f5": 5, "g5": 4, "h5": 3,
    "a4": 3, "b4": 4, "c4": 5, "d4": 6, "e4": 6, "f4": 5, "g4": 4, "h4": 3,
    "a3": 3, "b3": 4, "c3": 5, "d3": 6, "e3": 6, "f3": 5, "g3": 4, "h3": 3,
    "a2": 4, "b2": 5, "c2": 6, "d2": 7, "e2": 7, "f2": 6, "g2": 5, "h2": 4,
    "a1": 5, "b1": 6, "c1": 7, "d1": 8, "e1": 8, "f1": 7, "g1": 6, "h1": 5
}

board_map = {
    10: pawn_board_map,
    30: standard_board_map,
    35: standard_board_map,
    50: rook_board_map,
    90: standard_board_map,
    2000: king_board_map
}

def check_for_simple_threat(opponent_moves, cell, piece_value):
    # Checks if piece could be attacked by opponent's piece
    for move in opponent_moves:
        if cell == move[2:4]:
            return piece_value
    return 0

def find_move(game, fen):
    moves = game.get_moves()
    best_move = None
    best_value = -100

    for move in moves:
        piece_value = value_map[game.board.get_piece(game.xy2i(move[:2]))]
        capture_value = value_map[game.board.get_piece(game.xy2i(move[2:4]))]
        game.apply_move(move)

        if game.status == Game.CHECKMATE:
            return move

        # Get opponent's moves and calculate potential capture value
        opponent_moves = game.get_moves()
        netto_value = capture_value - check_for_simple_threat(opponent_moves, move[2:4], piece_value)

        if netto_value >= 0:
            netto_value += board_map[piece_value][move[2:4]] 
            if netto_value > best_value:
                best_value = netto_value 
                best_move = move
                if netto_value >= 10:  # Early exit if sufficient gain is found
                    return best_move

        # Reset game state after each move
        game.set_fen(fen)

    if best_move:
        return best_move
    return random.choice(moves)

def panic_mode(game, fen):
    # Aaaahhhh! Oh no, I'm running out of time!
    moves = game.get_moves()
    for move in moves:
        game.apply_move(move)
        if game.status == Game.CHECKMATE:
            return move
        game.set_fen(fen)
    return random.choice(moves)

def Chess_bot__Small_Snail__LennartHaupts(obs):
    game = Game(fen=obs.board)  # Intialize the game state with the current FEN
    if obs.remainingOverageTime < 2:
        return panic_mode(game, obs.board)

    # Find the "best" move using the evaluation function
    return find_move(game, obs.board)

Writing Chess_bot__Small_Snail__LennartHaupts.py


In [11]:
%%writefile Chess_bot__Start_with_strategic_move__FakeOrange.py

import random
from Chessnut import Game

# define piece values
PIECE_VALUES = {'p': 1, 'n': 3, 'b': 3, 'r': 5, 'q': 9, 'k': 50}

def evaluate_capture(game, move):
    """
    Evaluate the value of a capture move based on the target piece's value.

    Args:
        game: Current Game object.
        move: A move string in UCI format.

    Returns:
        Integer value representing the target piece's value.
    """
    target_square = move[2:4]  # Target position of the move
    target_piece = game.board.get_piece(Game.xy2i(target_square))
    if target_piece != ' ':
        return PIECE_VALUES[target_piece.lower()]
    return 0


def Chess_bot__Start_with_strategic_move__FakeOrange(obs):
    # 0. Parse the current board state and generate legal moves
    game = Game(obs.board)
    moves = list(game.get_moves())

    # 1. Check for checkmate
    for move in moves:
        g = Game(obs.board)
        g.apply_move(move)
        if g.status == Game.CHECKMATE:
            return move

    # 2. Check for valuable captures
    capture_moves = [(move, evaluate_capture(game, move)) for move in moves]
    capture_moves = [m for m in capture_moves if m[1] > 0]  # Filter only capture moves
    if capture_moves:
        return max(capture_moves, key=lambda x: x[1])[0]  # Return the move with the highest capture value

    # 3. Check for queen promotions
    promotion_moves = [move for move in moves if "q" in move.lower()]
    if promotion_moves:
        return random.choice(promotion_moves)

    # 4. Random move if no better options
    return random.choice(moves)

Writing Chess_bot__Start_with_strategic_move__FakeOrange.py


In [12]:
%%writefile Chess_bot__AGN_Cube4D_DRE__Callum_Maystone.py

import logging
from Chessnut import Game

# Configure Debugging and Logging
DEBUG = True
logging.basicConfig(level=logging.DEBUG if DEBUG else logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def debug(message):
    """Utility function for debugging."""
    if DEBUG:
        logging.debug(message)

# Constants
PIECE_VALUES = {
    "p": 1, "n": 3, "b": 3, "r": 5, "q": 9, "k": 0,
    "P": -1, "N": -3, "B": -3, "R": -5, "Q": -9, "K": 0
}
CENTER_SQUARES = {"d4", "d5", "e4", "e5"}

def load_weights():
    """Load weights for move prioritization."""
    # Optional: Extend with dynamic learning
    return {"central_control": 3, "captures": 5, "check": 10, "promotion": 15}

WEIGHTS = load_weights()

def parse_board_from_fen(fen):
    """
    Parse the board from the FEN string into a dictionary representation.
    Returns a dictionary where keys are square names (e.g., "a1") and values are pieces (e.g., "P", "r").
    """
    rows = fen.split()[0].split("/")
    board = {}
    for rank_idx, row in enumerate(rows):
        file_idx = 0
        for char in row:
            if char.isdigit():
                file_idx += int(char)  # Skip empty squares
            else:
                square = f"{chr(file_idx + ord('a'))}{8 - rank_idx}"
                board[square] = char
                file_idx += 1
    return board

def get_game_phase(game):
    """
    Determine the game phase: opening, midgame, or endgame.
    Based on material count.
    """
    fen = game.get_fen()
    board = parse_board_from_fen(fen)
    material_count = sum(abs(PIECE_VALUES[piece.lower()]) for piece in board.values() if piece != " ")
    if material_count > 30:
        return "opening"
    elif 20 < material_count <= 30:
        return "midgame"
    else:
        return "end"

def prioritize_moves(game, moves):
    """
    Prioritize moves based on first principles and game phase.
    """
    game_phase = get_game_phase(game)
    prioritized_moves = []

    for move in moves:
        piece = game.board.get_piece(Game.xy2i(move[:2]))
        target_square = move[2:4]
        target_piece = game.board.get_piece(Game.xy2i(target_square))

        # Checkmate moves (highest priority)
        g = Game(game.get_fen())
        g.apply_move(move)
        if g.status == Game.CHECKMATE:
            debug(f"Checkmate move found: {move}")
            return [move]

        # Endgame: Aggressively target the king
        if game_phase == "end":
            if g.status == Game.CHECK:
                debug(f"Check move in endgame: {move}")
                prioritized_moves.append((move, WEIGHTS["check"]))
            elif target_piece == 'K':  # Target the king
                debug(f"King-targeting move: {move}")
                prioritized_moves.append((move, WEIGHTS["check"]))
            continue

        # Captures (high priority)
        if target_piece != ' ':
            debug(f"Capture move: {move}, Captures: {target_piece}")
            prioritized_moves.append((move, WEIGHTS["captures"] + PIECE_VALUES.get(target_piece.lower(), 0)))

        # Central control (medium priority)
        elif target_square in CENTER_SQUARES:
            debug(f"Central control move: {move}")
            prioritized_moves.append((move, WEIGHTS["central_control"]))

        # Development of knights and bishops (early-game priority)
        elif piece.lower() in {'n', 'b'}:
            debug(f"Development move: {move}")
            prioritized_moves.append((move, 1))

        # Pawn promotion (endgame priority)
        elif piece.lower() == 'p':
            rank = int(target_square[1])
            if (rank == 8 and piece.islower()) or (rank == 1 and piece.isupper()):
                debug(f"Pawn promotion move: {move}")
                prioritized_moves.append((move, WEIGHTS["promotion"]))

        # Default: Other moves (low priority)
        else:
            prioritized_moves.append((move, 0))

    # Sort moves by priority (descending)
    prioritized_moves.sort(key=lambda x: x[1], reverse=True)
    return [move for move, _ in prioritized_moves]

def Chess_bot__AGN_Cube4D_DRE__Callum_Maystone(obs):
    """
    Enhanced chess bot using prioritized move evaluation.
    """
    game = Game(obs.board)
    moves = list(game.get_moves())

    if not moves:
        return None  # No legal moves available

    # Prioritize moves based on first principles
    prioritized_moves = prioritize_moves(game, moves)

    # Pick the best move (highest priority)
    best_move = prioritized_moves[0] if prioritized_moves else random.choice(moves)
    debug(f"Best move selected: {best_move}")
    return best_move

Writing Chess_bot__AGN_Cube4D_DRE__Callum_Maystone.py


In [13]:
%%writefile Chess_bot__Advanced_Engine_with_AB_Pruning__Shawn_Xu.py

from Chessnut import Game
from collections import Counter

# Basic piece values

MATERIAL_VALUE: dict[str, int] = {
    "P": 82,
    "N": 337,
    "B": 365,
    "R": 477,
    "Q": 1025,
    "K": 0,
    "p": -82,
    "n": -337,
    "b": -365,
    "r": -477,
    "q": -1025,
    "k": 0,
}

# Positional values from Rofchade:
# https://www.talkchess.com/forum3/viewtopic.php?f=2&t=68311&start=19

PAWN_VALUES: list[int] = [
      0,   0,   0,   0,   0,   0,  0,   0,
     98, 134,  61,  95,  68, 126, 34, -11,
     -6,   7,  26,  31,  65,  56, 25, -20,
    -14,  13,   6,  21,  23,  12, 17, -23,
    -27,  -2,  -5,  12,  17,   6, 10, -25,
    -26,  -4,  -4, -10,   3,   3, 33, -12,
    -35,  -1, -20, -23, -15,  24, 38, -22,
      0,   0,   0,   0,   0,   0,  0,   0,
]


KNIGHT_VALUES: list[int] = [
    -167, -89, -34, -49,  61, -97, -15, -107,
     -73, -41,  72,  36,  23,  62,   7,  -17,
     -47,  60,  37,  65,  84, 129,  73,   44,
      -9,  17,  19,  53,  37,  69,  18,   22,
     -13,   4,  16,  13,  28,  19,  21,   -8,
     -23,  -9,  12,  10,  19,  17,  25,  -16,
     -29, -53, -12,  -3,  -1,  18, -14,  -19,
    -105, -21, -58, -33, -17, -28, -19,  -23,
];

BISHOP_VALUES: list[int] = [
    -29,   4, -82, -37, -25, -42,   7,  -8,
    -26,  16, -18, -13,  30,  59,  18, -47,
    -16,  37,  43,  40,  35,  50,  37,  -2,
     -4,   5,  19,  50,  37,  37,   7,  -2,
     -6,  13,  13,  26,  34,  12,  10,   4,
      0,  15,  15,  15,  14,  27,  18,  10,
      4,  15,  16,   0,   7,  21,  33,   1,
    -33,  -3, -14, -21, -13, -12, -39, -21,
];

ROOK_VALUES: list[int] = [
     32,  42,  32,  51, 63,  9,  31,  43,
     27,  32,  58,  62, 80, 67,  26,  44,
     -5,  19,  26,  36, 17, 45,  61,  16,
    -24, -11,   7,  26, 24, 35,  -8, -20,
    -36, -26, -12,  -1,  9, -7,   6, -23,
    -45, -25, -16, -17,  3,  0,  -5, -33,
    -44, -16, -20,  -9, -1, 11,  -6, -71,
    -19, -13,   1,  17, 16,  7, -37, -26,
];

QUEEN_VALUES: list[int] = [
    -28,   0,  29,  12,  59,  44,  43,  45,
    -24, -39,  -5,   1, -16,  57,  28,  54,
    -13, -17,   7,   8,  29,  56,  47,  57,
    -27, -27, -16, -16,  -1,  17,  -2,   1,
     -9, -26,  -9, -10,  -2,  -4,   3,  -3,
    -14,   2, -11,  -2,  -5,   2,  14,   5,
    -35,  -8,  11,   2,   8,  15,  -3,   1,
     -1, -18,  -9,  10, -15, -25, -31, -50,
];

KING_VALUES: list[int] = [
    -65,  23,  16, -15, -56, -34,   2,  13,
     29,  -1, -20,  -7,  -8,  -4, -38, -29,
     -9,  24,   2, -16, -20,   6,  22, -22,
    -17, -20, -12, -27, -30, -25, -14, -36,
    -49,  -1, -27, -39, -46, -44, -33, -51,
    -14, -14, -22, -46, -44, -30, -15, -27,
      1,   7,  -8, -64, -43, -16,   9,   8,
    -15,  36,  12, -54,   8, -28,  24,  14,
];

POSITIONAL_VALUE: dict[str, list[int]] = {
    "P": PAWN_VALUES,
    "N": KNIGHT_VALUES,
    "B": BISHOP_VALUES,
    "R": ROOK_VALUES,
    "Q": QUEEN_VALUES,
    "K": KING_VALUES
}

### Board Evaluation ###
def evaluate(game: Game) -> int:
    eval: int = 0

    # Basic material eval.
    eval += sum(MATERIAL_VALUE.get(piece, 0) for piece in game.get_fen().split(" ")[0])

    # Positional eval
    for i in range(64):
        piece: str = game.board.get_piece(i)

        if piece.upper() not in POSITIONAL_VALUE:
            continue

        piece_positional_value: int = POSITIONAL_VALUE[piece.upper()][i]

        eval += piece_positional_value if piece.isupper() else -piece_positional_value

    # IMPORTANT: In negamax, board evaluation must be side-to-move-relative
    if game.state.player == "b":
        eval = -eval

    return eval


### Define Constants ###
# Maximum possible evaluation, acts as a root bound for alpha-beta pruning
MAX_EVAL: int = 10000000

# Evaluation when board is drawn. Can be adjusted to discourage or encourage draws
DRAW_EVAL: int = 0


### Threefold Detection ###
def is_threefold(fen_history: list[str]) -> bool:
    return Counter(fen_history).most_common(1)[0][1] >= 3


### Negamax Search ###
def search(
    depth: int, ply: int, game: Game, alpha: int, beta: int
) -> tuple[int, str | None]:
    # Evaluate when depth <= 0
    if depth <= 0:
        return evaluate(game), None

    # Check for three-fold repetition
    if is_threefold(game.fen_history):
        return DRAW_EVAL, None

    legal_moves: list[str] = game.get_moves()

    # Check for stalemate or checkmate. Code taken from Chessnut library
    # and simplified
    if len(legal_moves) == 0:
        # Calculate whether the king is in check
        us: str = game.state.player
        king_piece, them = {"w": ("K", "b"), "b": ("k", "w")}.get(us)
        king_location: str = Game.i2xy(game.board.find_piece(king_piece))
        in_check = bool(
            [m[2:] for m in game._all_moves(player=them) if m[2:] == king_location]
        )

        # Stalemate
        if not in_check:
            return DRAW_EVAL, None

        # Prioritize shorter mates
        return -MAX_EVAL + ply, None

    # TODO: Here, you can order moves by most promising in order
    # to reach a beta-cutoff faster should it happen.

    # Track best value and best move
    best_value: int = -MAX_EVAL
    best_move: str | None = None

    # Make a copy of the position to perform search
    original_fen: str = game.get_fen()
    fen_history_reference: list[str] = game.fen_history

    for move in legal_moves:
        # Make move
        game.apply_move(move)

        # Negamax search, result is negated to keep score side-relative
        value: int = -search(depth - 1, ply + 1, game, -beta, -alpha)[0]

        # Update best move
        if value > best_value:
            best_value = value
            best_move = move

        if best_value > alpha:
            alpha = best_value
            # Beta cutoff
            if alpha >= beta:
                break

        # This part is messy. Since Chessnut cannot automatically restore fen history
        # we have to restore it by hand
        game.set_fen(original_fen)
        game.fen_history = fen_history_reference
        fen_history_reference.pop()

    return best_value, best_move

def chess_bot__Advanced_Engine_with_AB_Pruning__Shawn_Xu(obs):
    game = Game(obs.board)

    # Limit search depth depending on time left
    # so we don't run out of time
    root_depth = 3 - (obs.remainingOverageTime < 2) - (obs.remainingOverageTime < 1)

    # Perform negamax search with alpha-beta pruning
    value, move = search(root_depth, 0, game, -MAX_EVAL, MAX_EVAL)
    return move      

Writing Chess_bot__Advanced_Engine_with_AB_Pruning__Shawn_Xu.py


In [14]:
%%writefile Chess_bot__Getting_under_the_hood_of_Chessnut__StuartRickard.py
from Chessnut import Game

def chess_bot__Getting_under_the_hood_of_Chessnut__StuartRickard(obs):
    """
    We will save ExtendedGame as part of the chess_bot function. This only needs to be done once.
    See the references at the bottom of this workbook for a link about how to do this.
    """
    if not hasattr(chess_bot__Getting_under_the_hood_of_Chessnut__StuartRickard, "extend"):
        """
        Define the ExtendedGame class
        """
        class ExtendedGame(Game): # We are creating a new class with the Game class as an argument
            def __init__(self, fen, validate=True):
                # This calls the parent class's initializer
                super().__init__(fen, validate)
                """
                Create a property of ExtendedGame that is a List. Here we'll store data about taking 
                an opponents piece. 
                """
                self.potential_takes = []

            # To redefine the _trace_ray method of Game, we restate its name
            def _trace_ray(self, start, piece, ray, player):

                res_moves = []
        
                for end in ray:
        
                    sym = piece.lower()
                    del_x = abs(end - start) % 8
                    move = [Game.i2xy(start) + Game.i2xy(end)]
                    tgt_owner = self.board.get_owner(end)
        
                    if tgt_owner == player:
                        break
        
                    if sym == 'k' and del_x == 2:
                        gap_owner = self.board.get_owner((start + end) // 2)
                        out_owner = self.board.get_owner(end - 1)
                        rights = {62: 'K', 58: 'Q', 6: 'k', 2: 'q'}.get(end, ' ')
                        if (tgt_owner or gap_owner or rights not in self.state.rights or
                                (rights.lower() == 'q' and out_owner)):
                            # Abort castling because missing castling rights
                            # or piece in the way
                            break
        
                    if sym == 'p':
                        if del_x == 0 and tgt_owner:
                            break
        
                        elif del_x != 0 and not tgt_owner:
                            ep_coords = self.state.en_passant
                            if ep_coords == '-' or end != Game.xy2i(ep_coords):
                                break
        
                        if (end < 8 or end > 55):
                            move = [move[0] + s for s in ['b', 'n', 'r', 'q']]
        
                    res_moves.extend(move)
        
                    # break after capturing an enemy piece
                    if tgt_owner:

                        """
                        All of Chessnut's _trace_ray code above is unchanged. At this conditional
                        statement, we add two lines of code to collect data about moves that could 
                        result in capturing an opponent's piece. Note "could", because these moves 
                        are not necessarily valid. To keep things simple, we'll handle validation  
                        outside of the Game class.
                        """
                        take_data = [move[-1], piece, self.board.get_piece(end)] # The move and the pieces
                        self.potential_takes.append(take_data) # Add to the 'potential_takes' property List
                        
                        break
        
                return res_moves        
        
            def get_potential_takes(self):
                return self.potential_takes

        chess_bot__Getting_under_the_hood_of_Chessnut__StuartRickard.extend = ExtendedGame
    
    """
    Instead of using Chessnut's Game here, we use the new ExtendedGame, which is stored at chess_bot.extend
    """
    game = chess_bot__Getting_under_the_hood_of_Chessnut__StuartRickard.extend(obs.board)
    moves = list(game.get_moves())
    
    """
    Validate takes
    """
    potential_takes = game.get_potential_takes()
    valid_takes = [take for take in potential_takes if take[0] in moves]

    # Sort valid_takes by value of opponent's piece
    def opponent_piece_value(take):
        return {'q': 9, 'r': 5, 'n': 3, 'b': 3, 'p': 1}[take[2].lower()]
    sorted_takes = sorted(valid_takes, key = opponent_piece_value, reverse = True)
    print(sorted_takes)

    # Take highest value piece if possible; otherwise just do the first move in the moves List
    if len(sorted_takes) > 0: return sorted_takes[0][0]
    return moves[0]

Writing Chess_bot__Getting_under_the_hood_of_Chessnut__StuartRickard.py


In [15]:
%%writefile Chess_bot__Your_First_Chess_Bot__ABHIJEET_SAMAL.py
from Chessnut import Game
import random

def chess_bot__Your_First_Chess_Bot__ABHIJEET_SAMAL(obs):
    """
    Simple chess bot that prioritizes checkmates, then captures, queen promotions, then randomly moves.

    Args:
        obs: An object with a 'board' attribute representing the current board state as a FEN string.

    Returns:
        A string representing the chosen move in UCI notation (e.g., "e2e4")
    """
    # 0. Parse the current board state and generate legal moves using Chessnut library
    game = Game(obs.board)
    moves = list(game.get_moves())

    # 1. Check a subset of moves for checkmate
    for move in moves[:10]:
        g = Game(obs.board)
        g.apply_move(move)
        if g.status == Game.CHECKMATE:
            return move

    # 2. Check for captures
    for move in moves:
        if game.board.get_piece(Game.xy2i(move[2:4])) != ' ':
            return move

    # 3. Check for queen promotions
    for move in moves:
        if "q" in move.lower():
            return move

    # 4. Pawn structure management
    for move in moves:
        if move[0] == 'P':  # pawn move
            # Avoid weak pawns
            if game.board.get_piece(Game.xy2i(move[2:4])) == 'P':
                continue
            # Create pawn chains
            if game.board.get_piece(Game.xy2i(move[2:4] + '1')) == 'P':
                return move

    # 5. Piece development
    for move in moves:
        if move[0] in ['N', 'B', 'R', 'Q']:
            # Develop pieces towards the center
            if abs(Game.xy2i(move[2:4]) - 4) < 3:
                return move

  # 6. Attack and defense
    for move in moves:
        # Identify weak points in opponent's position
        opponent_piece = game.board.get_piece(Game.xy2i(move[2:4]))
        if opponent_piece!='' and opponent_piece!= 'K':
            return move

     # 7. Endgame play
    for move in moves:
        # Promote pawns to queens or rooks
        if move[0] == 'P' and move[2] == '8':
            return move

    # 4. Random move if no checkmates or captures
    return random.choice(moves)

Writing Chess_bot__Your_First_Chess_Bot__ABHIJEET_SAMAL.py


In [16]:
%%writefile Chess_bot__Sunfish__Byfone.py

import time, math
from itertools import count
from collections import namedtuple

version = "sunfish 2023"

###############################################################################
# Piece-Square tables. Tune these to change sunfish's behaviour
###############################################################################

# With xz compression this whole section takes 652 bytes.
# That's pretty good given we have 64*6 = 384 values.
# Though probably we could do better...
# For one thing, they could easily all fit into int8.
piece = {"P": 100, "N": 280, "B": 320, "R": 479, "Q": 929, "K": 60000}
pst = {
    'P': (   0,   0,   0,   0,   0,   0,   0,   0,
            78,  83,  86,  73, 102,  82,  85,  90,
             7,  29,  21,  44,  40,  31,  44,   7,
           -17,  16,  -2,  15,  14,   0,  15, -13,
           -26,   3,  10,   9,   6,   1,   0, -23,
           -22,   9,   5, -11, -10,  -2,   3, -19,
           -31,   8,  -7, -37, -36, -14,   3, -31,
             0,   0,   0,   0,   0,   0,   0,   0),
    'N': ( -66, -53, -75, -75, -10, -55, -58, -70,
            -3,  -6, 100, -36,   4,  62,  -4, -14,
            10,  67,   1,  74,  73,  27,  62,  -2,
            24,  24,  45,  37,  33,  41,  25,  17,
            -1,   5,  31,  21,  22,  35,   2,   0,
           -18,  10,  13,  22,  18,  15,  11, -14,
           -23, -15,   2,   0,   2,   0, -23, -20,
           -74, -23, -26, -24, -19, -35, -22, -69),
    'B': ( -59, -78, -82, -76, -23,-107, -37, -50,
           -11,  20,  35, -42, -39,  31,   2, -22,
            -9,  39, -32,  41,  52, -10,  28, -14,
            25,  17,  20,  34,  26,  25,  15,  10,
            13,  10,  17,  23,  17,  16,   0,   7,
            14,  25,  24,  15,   8,  25,  20,  15,
            19,  20,  11,   6,   7,   6,  20,  16,
            -7,   2, -15, -12, -14, -15, -10, -10),
    'R': (  35,  29,  33,   4,  37,  33,  56,  50,
            55,  29,  56,  67,  55,  62,  34,  60,
            19,  35,  28,  33,  45,  27,  25,  15,
             0,   5,  16,  13,  18,  -4,  -9,  -6,
           -28, -35, -16, -21, -13, -29, -46, -30,
           -42, -28, -42, -25, -25, -35, -26, -46,
           -53, -38, -31, -26, -29, -43, -44, -53,
           -30, -24, -18,   5,  -2, -18, -31, -32),
    'Q': (   6,   1,  -8,-104,  69,  24,  88,  26,
            14,  32,  60, -10,  20,  76,  57,  24,
            -2,  43,  32,  60,  72,  63,  43,   2,
             1, -16,  22,  17,  25,  20, -13,  -6,
           -14, -15,  -2,  -5,  -1, -10, -20, -22,
           -30,  -6, -13, -11, -16, -11, -16, -27,
           -36, -18,   0, -19, -15, -15, -21, -38,
           -39, -30, -31, -13, -31, -36, -34, -42),
    'K': (   4,  54,  47, -99, -99,  60,  83, -62,
           -32,  10,  55,  56,  56,  55,  10,   3,
           -62,  12, -57,  44, -67,  28,  37, -31,
           -55,  50,  11,  -4, -19,  13,   0, -49,
           -55, -43, -52, -28, -51, -47,  -8, -50,
           -47, -42, -43, -79, -64, -32, -29, -32,
            -4,   3, -14, -50, -57, -18,  13,   4,
            17,  30,  -3, -14,   6,  -1,  40,  18),
}
# Pad tables and join piece and pst dictionaries
for k, table in pst.items():
    padrow = lambda row: (0,) + tuple(x + piece[k] for x in row) + (0,)
    pst[k] = sum((padrow(table[i * 8 : i * 8 + 8]) for i in range(8)), ())
    pst[k] = (0,) * 20 + pst[k] + (0,) * 20

###############################################################################
# Global constants
###############################################################################

# Our board is represented as a 120 character string. The padding allows for
# fast detection of moves that don't stay within the board.
A1, H1, A8, H8 = 91, 98, 21, 28
initial = (
    "         \n"  #   0 -  9
    "         \n"  #  10 - 19
    " rnbqkbnr\n"  #  20 - 29
    " pppppppp\n"  #  30 - 39
    " ........\n"  #  40 - 49
    " ........\n"  #  50 - 59
    " ........\n"  #  60 - 69
    " ........\n"  #  70 - 79
    " PPPPPPPP\n"  #  80 - 89
    " RNBQKBNR\n"  #  90 - 99
    "         \n"  # 100 -109
    "         \n"  # 110 -119
)

# Lists of possible moves for each piece type.
N, E, S, W = -10, 1, 10, -1
directions = {
    "P": (N, N+N, N+W, N+E),
    "N": (N+N+E, E+N+E, E+S+E, S+S+E, S+S+W, W+S+W, W+N+W, N+N+W),
    "B": (N+E, S+E, S+W, N+W),
    "R": (N, E, S, W),
    "Q": (N, E, S, W, N+E, S+E, S+W, N+W),
    "K": (N, E, S, W, N+E, S+E, S+W, N+W)
}

# Mate value must be greater than 8*queen + 2*(rook+knight+bishop)
# King value is set to twice this value such that if the opponent is
# 8 queens up, but we got the king, we still exceed MATE_VALUE.
# When a MATE is detected, we'll set the score to MATE_UPPER - plies to get there
# E.g. Mate in 3 will be MATE_UPPER - 6
MATE_LOWER = piece["K"] - 10 * piece["Q"]
MATE_UPPER = piece["K"] + 10 * piece["Q"]

# Constants for tuning search
QS = 40
QS_A = 140
EVAL_ROUGHNESS = 15

# minifier-hide start
opt_ranges = dict(
    QS = (0, 300),
    QS_A = (0, 300),
    EVAL_ROUGHNESS = (0, 50),
)
# minifier-hide end


###############################################################################
# Chess logic
###############################################################################


Move = namedtuple("Move", "i j prom")


class Position(namedtuple("Position", "board score wc bc ep kp")):
    """A state of a chess game
    board -- a 120 char representation of the board
    score -- the board evaluation
    wc -- the castling rights, [west/queen side, east/king side]
    bc -- the opponent castling rights, [west/king side, east/queen side]
    ep - the en passant square
    kp - the king passant square
    """

    def gen_moves(self):
        # For each of our pieces, iterate through each possible 'ray' of moves,
        # as defined in the 'directions' map. The rays are broken e.g. by
        # captures or immediately in case of pieces such as knights.
        for i, p in enumerate(self.board):
            if not p.isupper():
                continue
            for d in directions[p]:
                for j in count(i + d, d):
                    q = self.board[j]
                    # Stay inside the board, and off friendly pieces
                    if q.isspace() or q.isupper():
                        break
                    # Pawn move, double move and capture
                    if p == "P":
                        if d in (N, N + N) and q != ".": break
                        if d == N + N and (i < A1 + N or self.board[i + N] != "."): break
                        if (
                            d in (N + W, N + E)
                            and q == "."
                            and j not in (self.ep, self.kp, self.kp - 1, self.kp + 1)
                            #and j != self.ep and abs(j - self.kp) >= 2
                        ):
                            break
                        # If we move to the last row, we can be anything
                        if A8 <= j <= H8:
                            for prom in "NBRQ":
                                yield Move(i, j, prom)
                            break
                    # Move it
                    yield Move(i, j, "")
                    # Stop crawlers from sliding, and sliding after captures
                    if p in "PNK" or q.islower():
                        break
                    # Castling, by sliding the rook next to the king
                    if i == A1 and self.board[j + E] == "K" and self.wc[0]:
                        yield Move(j + E, j + W, "")
                    if i == H1 and self.board[j + W] == "K" and self.wc[1]:
                        yield Move(j + W, j + E, "")

    def rotate(self, nullmove=False):
        """Rotates the board, preserving enpassant, unless nullmove"""
        return Position(
            self.board[::-1].swapcase(), -self.score, self.bc, self.wc,
            119 - self.ep if self.ep and not nullmove else 0,
            119 - self.kp if self.kp and not nullmove else 0,
        )

    def move(self, move):
        i, j, prom = move
        p, q = self.board[i], self.board[j]
        put = lambda board, i, p: board[:i] + p + board[i + 1 :]
        # Copy variables and reset ep and kp
        board = self.board
        wc, bc, ep, kp = self.wc, self.bc, 0, 0
        score = self.score + self.value(move)
        # Actual move
        board = put(board, j, board[i])
        board = put(board, i, ".")
        # Castling rights, we move the rook or capture the opponent's
        if i == A1: wc = (False, wc[1])
        if i == H1: wc = (wc[0], False)
        if j == A8: bc = (bc[0], False)
        if j == H8: bc = (False, bc[1])
        # Castling
        if p == "K":
            wc = (False, False)
            if abs(j - i) == 2:
                kp = (i + j) // 2
                board = put(board, A1 if j < i else H1, ".")
                board = put(board, kp, "R")
        # Pawn promotion, double move and en passant capture
        if p == "P":
            if A8 <= j <= H8:
                board = put(board, j, prom)
            if j - i == 2 * N:
                ep = i + N
            if j == self.ep:
                board = put(board, j + S, ".")
        # We rotate the returned position, so it's ready for the next player
        return Position(board, score, wc, bc, ep, kp).rotate()

    def value(self, move):
        i, j, prom = move
        p, q = self.board[i], self.board[j]
        # Actual move
        score = pst[p][j] - pst[p][i]
        # Capture
        if q.islower():
            score += pst[q.upper()][119 - j]
        # Castling check detection
        if abs(j - self.kp) < 2:
            score += pst["K"][119 - j]
        # Castling
        if p == "K" and abs(i - j) == 2:
            score += pst["R"][(i + j) // 2]
            score -= pst["R"][A1 if j < i else H1]
        # Special pawn stuff
        if p == "P":
            if A8 <= j <= H8:
                score += pst[prom][j] - pst["P"][j]
            if j == self.ep:
                score += pst["P"][119 - (j + S)]
        return score


###############################################################################
# Search logic
###############################################################################

# lower <= s(pos) <= upper
Entry = namedtuple("Entry", "lower upper")


class Searcher:
    def __init__(self):
        self.tp_score = {}
        self.tp_move = {}
        self.history = set()
        self.nodes = 0

    def bound(self, pos, gamma, depth, can_null=True):
        """ Let s* be the "true" score of the sub-tree we are searching.
            The method returns r, where
            if gamma >  s* then s* <= r < gamma  (A better upper bound)
            if gamma <= s* then gamma <= r <= s* (A better lower bound) """
        self.nodes += 1

        # Depth <= 0 is QSearch. Here any position is searched as deeply as is needed for
        # calmness, and from this point on there is no difference in behaviour depending on
        # depth, so so there is no reason to keep different depths in the transposition table.
        depth = max(depth, 0)

        # Sunfish is a king-capture engine, so we should always check if we
        # still have a king. Notice since this is the only termination check,
        # the remaining code has to be comfortable with being mated, stalemated
        # or able to capture the opponent king.
        if pos.score <= -MATE_LOWER:
            return -MATE_UPPER

        # Look in the table if we have already searched this position before.
        # We also need to be sure, that the stored search was over the same
        # nodes as the current search.
        entry = self.tp_score.get((pos, depth, can_null), Entry(-MATE_UPPER, MATE_UPPER))
        if entry.lower >= gamma: return entry.lower
        if entry.upper < gamma: return entry.upper

        # Let's not repeat positions. We don't chat
        # - at the root (can_null=False) since it is in history, but not a draw.
        # - at depth=0, since it would be expensive and break "futulity pruning".
        if can_null and depth > 0 and pos in self.history:
            return 0

        # Generator of moves to search in order.
        # This allows us to define the moves, but only calculate them if needed.
        def moves():
            # First try not moving at all. We only do this if there is at least one major
            # piece left on the board, since otherwise zugzwangs are too dangerous.
            # FIXME: We also can't null move if we can capture the opponent king.
            # Since if we do, we won't spot illegal moves that could lead to stalemate.
            # For now we just solve this by not using null-move in very unbalanced positions.
            # TODO: We could actually use null-move in QS as well. Not sure it would be very useful.
            # But still.... We just have to move stand-pat to be before null-move.
            #if depth > 2 and can_null and any(c in pos.board for c in "RBNQ"):
            #if depth > 2 and can_null and any(c in pos.board for c in "RBNQ") and abs(pos.score) < 500:
            if depth > 2 and can_null and abs(pos.score) < 500:
                yield None, -self.bound(pos.rotate(nullmove=True), 1 - gamma, depth - 3)

            # For QSearch we have a different kind of null-move, namely we can just stop
            # and not capture anything else.
            if depth == 0:
                yield None, pos.score

            # Look for the strongest ove from last time, the hash-move.
            killer = self.tp_move.get(pos)

            # If there isn't one, try to find one with a more shallow search.
            # This is known as Internal Iterative Deepening (IID). We set
            # can_null=True, since we want to make sure we actually find a move.
            if not killer and depth > 2:
                self.bound(pos, gamma, depth - 3, can_null=False)
                killer = self.tp_move.get(pos)

            # If depth == 0 we only try moves with high intrinsic score (captures and
            # promotions). Otherwise we do all moves. This is called quiescent search.
            val_lower = QS - depth * QS_A

            # Only play the move if it would be included at the current val-limit,
            # since otherwise we'd get search instability.
            # We will search it again in the main loop below, but the tp will fix
            # things for us.
            if killer and pos.value(killer) >= val_lower:
                yield killer, -self.bound(pos.move(killer), 1 - gamma, depth - 1)

            # Then all the other moves
            for val, move in sorted(((pos.value(m), m) for m in pos.gen_moves()), reverse=True):
                # Quiescent search
                if val < val_lower:
                    break

                # If the new score is less than gamma, the opponent will for sure just
                # stand pat, since ""pos.score + val < gamma === -(pos.score + val) >= 1-gamma""
                # This is known as futility pruning.
                if depth <= 1 and pos.score + val < gamma:
                    # Need special case for MATE, since it would normally be caught
                    # before standing pat.
                    yield move, pos.score + val if val < MATE_LOWER else MATE_UPPER
                    # We can also break, since we have ordered the moves by value,
                    # so it can't get any better than this.
                    break

                yield move, -self.bound(pos.move(move), 1 - gamma, depth - 1)

        # Run through the moves, shortcutting when possible
        best = -MATE_UPPER
        for move, score in moves():
            best = max(best, score)
            if best >= gamma:
                # Save the move for pv construction and killer heuristic
                if move is not None:
                    self.tp_move[pos] = move
                break

        # Stalemate checking is a bit tricky: Say we failed low, because
        # we can't (legally) move and so the (real) score is -infty.
        # At the next depth we are allowed to just return r, -infty <= r < gamma,
        # which is normally fine.
        # However, what if gamma = -10 and we don't have any legal moves?
        # Then the score is actaully a draw and we should fail high!
        # Thus, if best < gamma and best < 0 we need to double check what we are doing.

        # We will fix this problem another way: We add the requirement to bound, that
        # it always returns MATE_UPPER if the king is capturable. Even if another move
        # was also sufficient to go above gamma. If we see this value we know we are either
        # mate, or stalemate. It then suffices to check whether we're in check.

        # Note that at low depths, this may not actually be true, since maybe we just pruned
        # all the legal moves. So sunfish may report "mate", but then after more search
        # realize it's not a mate after all. That's fair.

        # This is too expensive to test at depth == 0
        if depth > 2 and best == -MATE_UPPER:
            flipped = pos.rotate(nullmove=True)
            # Hopefully this is already in the TT because of null-move
            in_check = self.bound(flipped, MATE_UPPER, 0) == MATE_UPPER
            best = -MATE_LOWER if in_check else 0

        # Table part 2
        if best >= gamma:
            self.tp_score[pos, depth, can_null] = Entry(best, entry.upper)
        if best < gamma:
            self.tp_score[pos, depth, can_null] = Entry(entry.lower, best)

        return best

    def search(self, history):
        """Iterative deepening MTD-bi search"""
        self.nodes = 0
        self.history = set(history)
        self.tp_score.clear()

        gamma = 0
        # In finished games, we could potentially go far enough to cause a recursion
        # limit exception. Hence we bound the ply. We also can't start at 0, since
        # that's quiscent search, and we don't always play legal moves there.
        for depth in range(1, 1000):
            # The inner loop is a binary search on the score of the position.
            # Inv: lower <= score <= upper
            # 'while lower != upper' would work, but it's too much effort to spend
            # on what's probably not going to change the move played.
            lower, upper = -MATE_LOWER, MATE_LOWER
            while lower < upper - EVAL_ROUGHNESS:
                score = self.bound(history[-1], gamma, depth, can_null=False)
                if score >= gamma:
                    lower = score
                if score < gamma:
                    upper = score
                yield depth, gamma, score, self.tp_move.get(history[-1])
                gamma = (lower + upper + 1) // 2


###############################################################################
# UCI User interface
###############################################################################

__debug = False

def parse(c):
    fil, rank = ord(c[0]) - ord("a"), int(c[1]) - 1
    return A1 + fil - 10 * rank

def render(i):
    rank, fil = divmod(i - A1, 10)
    return chr(fil + ord("a")) + str(-rank + 1)

def fen_to_position(fen):
    places, my_color, castling, en_passant, hm_clock, fm_number = fen.split()

    blank_lines = "         \n" * 2
    pos = blank_lines + " "
    for c in places:
        if c == "/":
            pos += "\n "
            continue
        try:
            pos += "." * int(c)
        except ValueError:
            pos += c
    pos += "\n" + blank_lines
    
    white_castling = (("Q" in castling), ("K" in castling))
    black_castling = (("k" in castling), ("q" in castling))

    en_passant = 0 if en_passant == "-" else parse(en_passant)
    position = Position(pos, 0, white_castling, black_castling, en_passant, 0)

    if my_color == "b":
        position = position.rotate()

    if __debug: print("Loaded position")

    return my_color, position


###############################################################################
# Chess Bot
###############################################################################


from Chessnut import Game
import random

def chess_bot__Sunfish__Byfone(obs):
    game = Game(obs.board)
    moves = list(game.get_moves())

    think = 0.1
    start = time.time()
    move_str = None
    my_color, position = fen_to_position(obs.board)
    hist = [position]
    for depth, gamma, score, move in Searcher().search(hist):
        # The only way we can be sure to have the real move in tp_move,
        # is if we have just failed high.
        if (score >= gamma) and move:
            i, j = move.i, move.j
            if my_color == "b":
                i, j = 119 - i, 119 - j
            move_str = render(i) + render(j) + move.prom.lower()
            if __debug: print("info depth", depth, "score cp", score, "pv", move_str)
        if move_str and time.time() - start > think * 0.8:
            break

    if move_str in moves:
        return move_str
    else:
        print("illegal move:", move_str)
        return random.choice(moves)

Writing Chess_bot__Sunfish__Byfone.py
